In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as mp
import matplotlib.ticker as mtick

In [2]:
# sensor: LoRaWAN, TI, Lansen
def getData(sensor):
    secondsPerMessage = 15 if sensor == 'TI' else 60
    
    print()
    print('Sensor: ', sensor)
    
    
    # Daten importieren
    if sensor == 'LoRaWAN':
        data = pd.read_csv('../Messwerte/Zuverlaessigkeit/Zuverlaessigkeitstest_lorawan_log.csv', skiprows=1,  sep=',', names=['gateway_ID', 'node_MAC', 'time', 'us_count', 'frequency', 'RF_chain', 'RX_chain', 'status', 'size', 'modulation', 'bandwidth', 'datarate', 'coderate', 'rssi', 'snr', 'frame'])
    else: # Hier liegen die Daten von beiden Mioty-Sensoren:
        data = pd.read_csv('../Messwerte/Zuverlaessigkeit/Zuverlaessigkeitstest_mioty_log.csv', names=['time', 'sensor', 'frame', 'rssi', 'snr', 'raw'], sep=';')
        
    # Bei Mioty werden die Daten des ausgewählten Sensors gefiltert
    if sensor != 'LoRaWAN':
        data = data[data['sensor'] == sensor]
        data = data.reset_index(drop=True)
    else:
        data = data[data['frame'].str.startswith('40C2FF0B-26')]
        # Doppelt empfangene Nachrichten verwerfen, da diese ungültig sind (Replay-Attack). Bei Mioty sind diese bereits durch das Gateway herausgefiltert
        data.drop_duplicates(subset=['frame'], inplace=True, ignore_index=True)

    data['time'] = pd.to_datetime(data['time'].str.replace(' ', 'T'), utc=True)

    # Der TI-Client sendet die Nachricht immer in zwei Teilen im Abstand von ca. 4 Sekunden. Damit diese Teile jeweils als einzelne Nachrichten
    # betrachtet werden können, wird zur Uhrzeit des zweiten Teils der Nachricht 11 Sekunden aufaddiert. So ergibt sich im Idealfall ein Abstand
    # von 15 Sekunden zwischen den Nachrichten
    if sensor == 'TI':
        data.loc[data['frame'].str.startswith('02'), 'time'] = data['time'][data['frame'].str.startswith('02')] + pd.Timedelta(seconds=11)

    # Abstand zwischen den Nachrichten
    data['time_delta'] = data['time'].diff().dt.total_seconds()

    # Bei Mioty wurden nicht empfangene Daten nicht aufgezeichnet. Bei LoRaWAN wurde aufgezeichnet, wenn sie von einem anderen TTN-Gateway
    # empfangen wurden. Received wird dann auf False gesetzt
    if sensor == 'LoRaWAN':
        data['received'] = data['status'].str.strip() == 'CRC_OK'
    else:
        data['received'] = True
        
        
    # Vom zeitliche Abstand jeder Nachricht wird der Soll-Abstand subtrahiert. Dadurch ergibt sich die Abweichung zum Soll.
    # Dann wird durch den Soll-Abstand dividiert und gerundet. So ergibt sich die Anzahl an Nachrichten, die jeweils zwischen zwei Nachrichten fehlen
    missingMessages = ((data['time_delta'][1:] - secondsPerMessage) / secondsPerMessage).round().astype(int)

    # Gibt nur die Anahl Nachrichten an, die gar nicht angekommen sind. Nachrichten dessen CRC fehlgeschlagen ist, werden hier nicht mitgezählt, 
    # sondern nur am Ende bei den nicht empfangenen Nachrichten
    print(missingMessages.sum(), 'messages missing')

    # Diese Anzahl an Nachrichten wird dann erzeugt mit dem Attribut received auf False
    insertedMessages = pd.DataFrame(data=[], columns=['time', 'received'])
    for index, val in missingMessages.iloc[::-1].items():
        time = data['time'][index - 1]
        for x in range(val):
            time = time + pd.Timedelta(seconds=secondsPerMessage)
            insertedMessages = insertedMessages.append({
                'time': time,
                'received': False
            }, ignore_index=True)

    # Neue Einträge hinzufügen und nach Uhrzeit sortieren
    newData = pd.concat([data, insertedMessages], ignore_index=True)
    newData = newData.sort_values('time', ignore_index=True)
    
    # Durchschnittliche SNR und RRSI berechnen
    print('snr: ', newData['snr'].mean())
    print('rssi: ', newData['rssi'].mean())
    # Anzahl an gesendeten und empfangen Nachrichten, sowie die Erfolgsrate berechnen
    sendCount = len(newData.index)
    successCount = (newData['received'] == True).sum()
    successRate = successCount / sendCount
    
    print('Sent: ', sendCount)
    print('Received: ', successCount)
    print('Rate: ', successRate)
    

In [3]:
getData('LoRaWAN')
getData('TI')
getData('Lansen')



Sensor:  LoRaWAN
1 messages missing
snr:  9.155014326647564
rssi:  -43.656160458452725
Sent:  350
Received:  347
Rate:  0.9914285714285714

Sensor:  TI
0 messages missing
snr:  24.178214285714287
rssi:  -56.78785714285714
Sent:  1400
Received:  1400
Rate:  1.0

Sensor:  Lansen
2 messages missing
snr:  24.45203488372093
rssi:  -57.63720930232558
Sent:  346
Received:  344
Rate:  0.9942196531791907
